In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading
import gc
import copy

sys.path.append("/end/home/rh2310/morpho_repo")
from turing.utils import *
import turing.pinns as tu
from turing.loss_functions import *

GPU = False
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

2022-07-06 00:50:53.244804: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1892] Ignoring visible gpu device (device: 0, name: Quadro K2000, pci bus id: 0000:05:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.


# Load data

In [2]:
data_path = os.path.abspath("turing.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("turing_t.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
T = t_star.shape[0]    
    
L = 50
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size

model_params = {'training_data_size': T*N,#T*32,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T)//(8)}

dataset = create_dataset(data, t_star, N, T, L, **model_params)
lb = dataset['lb']
ub = dataset['ub']
obs_X = dataset['obs_input']
obs_Y = dataset['obs_output']
pde_X = dataset['pde']


In [3]:
data_path = os.path.abspath("turing_model_2.npy")
with open(data_path, 'rb') as f:
    data2 = np.load(f)
    
data_path = os.path.abspath("turing_t_model_2.npy")
with open(data_path, 'rb') as f:
    t_star2 = np.load(f) 
    
T2 = t_star2.shape[0]    
    
L2 = 50
x_size2 = data2.shape[1]
y_size2 = data2.shape[2]
N2 = x_size2*y_size2

model_params2 = {'training_data_size': T2*N2,#T*32,
                'pde_data_size': (T2*N2)//(32),
                'boundary_data_size':((x_size2 + y_size2)*T2)//(8)}

dataset2 = create_dataset(data2, t_star2, N2, T2, L2, **model_params2)
lb2 = dataset['lb']
ub2 = dataset['ub']
obs_X2 = dataset2['obs_input']
obs_Y2 = dataset2['obs_output']
pde_X2 = dataset2['pde']

# Test 

In [10]:
layers = [3, 64, 64, 64, 64, 2]

pinn = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss = ASDM(dtype=tf.float64, D_a = 0.005, D_s = 0.2)

In [11]:
model = tu.TINN(pinn, pde_loss, alpha = 0.25)

In [12]:
results = model.train(epochs = 500,
                      batch_size = 512,
                      X = obs_X,
                      Y = obs_Y,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 4.0110
total loss: 25540.56468, total regularisd loss: 25540.56468
obs u loss: 583.38952, obs v loss: 13697.76128
pde u loss: 451.09257, pde v loss: 10808.32131
lambda obs u: 1.00000, lambda obs v: 1.00000
lambda pde u: 1.00000, lambda pde v: 1.00000
sigma_a:0:10.02714023, sigma_s:0:8.43172385, mu_a:0:9.79288470, rho_a:0:10.38194313, rho_s:0:11.83654228, kappa_a:0:8.04983544
Time taken: 48.70s

Start of epoch 1
Training observations acc over epoch: 4.1434
total loss: 11816.42264, total regularisd loss: 11816.42264
obs u loss: 592.33718, obs v loss: 7893.39371
pde u loss: 0.69590, pde v loss: 3329.99585
lambda obs u: 0.33306, lambda obs v: 2.75501
lambda pde u: 0.25000, lambda pde v: 0.66193
sigma_a:0:10.07004369, sigma_s:0:7.61199094, mu_a:0:9.69885849, rho_a:0:10.54549089, rho_s:0:12.64843414, kappa_a:0:7.09827691
Time taken: 32.78s

Start of epoch 2
Training observations acc over epoch: 0.9685
total loss: 13277.59580, total regu

Training observations acc over epoch: 0.1325
total loss: 24675.35869, total regularisd loss: 977.09384
obs u loss: 247.85191, obs v loss: 23.48482
pde u loss: 54.84619, pde v loss: 12.77024
lambda obs u: 3.94321, lambda obs v: 0.05153
lambda pde u: 0.00439, lambda pde v: 0.00088
sigma_a:0:8.36270425, sigma_s:0:4.27665489, mu_a:0:10.93609371, rho_a:0:9.22655265, rho_s:0:15.39068991, kappa_a:0:3.57435712
Time taken: 24.92s

Start of epoch 20
Training observations acc over epoch: 0.1326
total loss: 28819.25539, total regularisd loss: 978.60864
obs u loss: 247.80720, obs v loss: 23.67610
pde u loss: 50.34671, pde v loss: 14.75007
lambda obs u: 3.94620, lambda obs v: 0.04862
lambda pde u: 0.00413, lambda pde v: 0.00105
sigma_a:0:8.33229327, sigma_s:0:4.28410853, mu_a:0:10.91246189, rho_a:0:9.25006400, rho_s:0:15.37945116, kappa_a:0:3.55830894
Time taken: 24.98s

Start of epoch 21
Training observations acc over epoch: 0.1325
total loss: 25656.39001, total regularisd loss: 979.38681
obs u los

Training observations acc over epoch: 0.1324
total loss: 25447.83835, total regularisd loss: 977.57811
obs u loss: 247.77741, obs v loss: 23.38252
pde u loss: 43.12052, pde v loss: 13.69401
lambda obs u: 3.93797, lambda obs v: 0.05714
lambda pde u: 0.00395, lambda pde v: 0.00094
sigma_a:0:7.58793032, sigma_s:0:4.51174167, mu_a:0:10.55331915, rho_a:0:9.53890602, rho_s:0:14.35755646, kappa_a:0:3.04791491
Time taken: 24.90s

Start of epoch 39
Training observations acc over epoch: 0.1324
total loss: 26151.37664, total regularisd loss: 977.22928
obs u loss: 247.77072, obs v loss: 23.41013
pde u loss: 41.62934, pde v loss: 14.21471
lambda obs u: 3.93549, lambda obs v: 0.05946
lambda pde u: 0.00400, lambda pde v: 0.00105
sigma_a:0:7.52816816, sigma_s:0:4.51336087, mu_a:0:10.54475719, rho_a:0:9.54455685, rho_s:0:14.29971778, kappa_a:0:3.02579005
Time taken: 24.79s

Start of epoch 40
Training observations acc over epoch: 0.1324
total loss: 24956.81836, total regularisd loss: 976.92826
obs u los

Training observations acc over epoch: 0.1323
total loss: 25402.67214, total regularisd loss: 975.54613
obs u loss: 247.74017, obs v loss: 23.18069
pde u loss: 35.86446, pde v loss: 13.90504
lambda obs u: 3.92966, lambda obs v: 0.06577
lambda pde u: 0.00359, lambda pde v: 0.00098
sigma_a:0:6.62133533, sigma_s:0:4.74407639, mu_a:0:10.22276767, rho_a:0:9.77568235, rho_s:0:13.05733625, kappa_a:0:2.49188647
Time taken: 25.07s

Start of epoch 58
Training observations acc over epoch: 0.1323
total loss: 25052.48681, total regularisd loss: 975.17073
obs u loss: 247.73216, obs v loss: 23.18650
pde u loss: 35.50087, pde v loss: 14.43203
lambda obs u: 3.92873, lambda obs v: 0.06655
lambda pde u: 0.00369, lambda pde v: 0.00102
sigma_a:0:6.56345734, sigma_s:0:4.76291395, mu_a:0:10.20405374, rho_a:0:9.78734369, rho_s:0:12.97566292, kappa_a:0:2.46083428
Time taken: 25.34s

Start of epoch 59
Training observations acc over epoch: 0.1323
total loss: 23878.21453, total regularisd loss: 975.03533
obs u los

Training observations acc over epoch: 0.1322
total loss: 24201.74505, total regularisd loss: 973.53311
obs u loss: 247.71885, obs v loss: 22.98820
pde u loss: 29.72738, pde v loss: 14.49263
lambda obs u: 3.92148, lambda obs v: 0.07453
lambda pde u: 0.00300, lambda pde v: 0.00099
sigma_a:0:5.60000047, sigma_s:0:5.02807642, mu_a:0:9.87727249, rho_a:0:9.94916526, rho_s:0:11.64387537, kappa_a:0:1.94501307
Time taken: 24.96s

Start of epoch 77
Training observations acc over epoch: 0.1322
total loss: 24763.56869, total regularisd loss: 973.24033
obs u loss: 247.71910, obs v loss: 22.97236
pde u loss: 29.39977, pde v loss: 14.40444
lambda obs u: 3.91814, lambda obs v: 0.07784
lambda pde u: 0.00303, lambda pde v: 0.00099
sigma_a:0:5.53205759, sigma_s:0:5.03771576, mu_a:0:9.86639717, rho_a:0:9.94801966, rho_s:0:11.57219932, kappa_a:0:1.92284500
Time taken: 24.91s

Start of epoch 78
Training observations acc over epoch: 0.1322
total loss: 24882.84947, total regularisd loss: 972.59288
obs u loss:

...

Training observations acc over epoch: 0.0021
total loss: 8.09959, total regularisd loss: 9.29401
obs u loss: 2.79869, obs v loss: 1.49487
pde u loss: 3.79881, pde v loss: 0.39463
lambda obs u: 1.39637, lambda obs v: 1.09495
lambda pde u: 0.86113, lambda pde v: 0.64755
sigma_a:0:0.85651891, sigma_s:0:0.01882502, mu_a:0:4.49773412, rho_a:0:3.90176546, rho_s:0:0.02072651, kappa_a:0:0.30793386
Time taken: 25.84s

Start of epoch 419
Training observations acc over epoch: 0.0020
total loss: 8.20686, total regularisd loss: 8.77071
obs u loss: 2.62079, obs v loss: 1.46215
pde u loss: 3.78360, pde v loss: 0.38911
lambda obs u: 1.34286, lambda obs v: 1.10300
lambda pde u: 0.89013, lambda pde v: 0.66401
sigma_a:0:0.84275832, sigma_s:0:0.01954047, mu_a:0:4.44939684, rho_a:0:3.86687156, rho_s:0:0.02047024, kappa_a:0:0.30789488
Time taken: 25.50s

Start of epoch 420
Training observations acc over epoch: 0.0020
total loss: 7.87894, total regularisd loss: 8.56023
obs u loss: 2.60808, obs v loss: 1.4521

Training observations acc over epoch: 0.0014
total loss: 5.68205, total regularisd loss: 5.75262
obs u loss: 1.73465, obs v loss: 1.18158
pde u loss: 2.39084, pde v loss: 0.32538
lambda obs u: 1.04225, lambda obs v: 1.24249
lambda pde u: 0.95471, lambda pde v: 0.76055
sigma_a:0:0.66047178, sigma_s:0:0.01904837, mu_a:0:3.51208497, rho_a:0:3.03861728, rho_s:0:0.02057769, kappa_a:0:0.30174175
Time taken: 25.67s

Start of epoch 438
Training observations acc over epoch: 0.0014
total loss: 5.37979, total regularisd loss: 5.65929
obs u loss: 1.77742, obs v loss: 1.17904
pde u loss: 2.19390, pde v loss: 0.32515
lambda obs u: 1.12741, lambda obs v: 1.23981
lambda pde u: 0.86356, lambda pde v: 0.76921
sigma_a:0:0.65511563, sigma_s:0:0.02020321, mu_a:0:3.45701133, rho_a:0:2.98310303, rho_s:0:0.02002341, kappa_a:0:0.30121278
Time taken: 25.36s

Start of epoch 439
Training observations acc over epoch: 0.0014
total loss: 5.46888, total regularisd loss: 5.52394
obs u loss: 1.66424, obs v loss: 1.1494

Training observations acc over epoch: 0.0010
total loss: 3.45890, total regularisd loss: 3.67495
obs u loss: 1.18959, obs v loss: 0.84471
pde u loss: 1.24471, pde v loss: 0.24875
lambda obs u: 0.92237, lambda obs v: 1.21157
lambda pde u: 1.07030, lambda pde v: 0.79575
sigma_a:0:0.41118411, sigma_s:0:0.01950168, mu_a:0:2.17720315, rho_a:0:1.84972030, rho_s:0:0.01956139, kappa_a:0:0.28529803
Time taken: 25.60s

Start of epoch 457
Training observations acc over epoch: 0.0009
total loss: 3.33992, total regularisd loss: 3.52867
obs u loss: 1.11294, obs v loss: 0.79622
pde u loss: 1.25916, pde v loss: 0.23848
lambda obs u: 0.84071, lambda obs v: 1.17142
lambda pde u: 1.19949, lambda pde v: 0.78839
sigma_a:0:0.39079614, sigma_s:0:0.01944574, mu_a:0:2.08646089, rho_a:0:1.76843598, rho_s:0:0.01965643, kappa_a:0:0.28397208
Time taken: 25.54s

Start of epoch 458
Training observations acc over epoch: 0.0009
total loss: 3.25047, total regularisd loss: 3.40188
obs u loss: 1.13256, obs v loss: 0.7850

Training observations acc over epoch: 0.0003
total loss: 0.74971, total regularisd loss: 1.01846
obs u loss: 0.47581, obs v loss: 0.18024
pde u loss: 0.00702, pde v loss: 0.09741
lambda obs u: 1.48163, lambda obs v: 0.64938
lambda pde u: 0.03528, lambda pde v: 1.83371
sigma_a:0:0.00797549, sigma_s:0:0.01803798, mu_a:0:0.06309769, rho_a:0:0.06083584, rho_s:0:0.02018163, kappa_a:0:0.33784677
Time taken: 25.67s

Start of epoch 476
Training observations acc over epoch: 0.0003
total loss: 0.77869, total regularisd loss: 0.97688
obs u loss: 0.46472, obs v loss: 0.17499
pde u loss: 0.00507, pde v loss: 0.09517
lambda obs u: 1.51512, lambda obs v: 0.66518
lambda pde u: 0.02159, lambda pde v: 1.79812
sigma_a:0:0.00533539, sigma_s:0:0.01851507, mu_a:0:0.04868485, rho_a:0:0.04887603, rho_s:0:0.02125392, kappa_a:0:0.36082856
Time taken: 25.35s

Start of epoch 477
Training observations acc over epoch: 0.0003
total loss: 0.78251, total regularisd loss: 0.99847
obs u loss: 0.47200, obs v loss: 0.1714

Training observations acc over epoch: 0.0002
total loss: 1.58636, total regularisd loss: 0.76331
obs u loss: 0.35860, obs v loss: 0.13251
pde u loss: 0.00073, pde v loss: 0.08130
lambda obs u: 1.45200, lambda obs v: 0.54912
lambda pde u: 0.00069, lambda pde v: 1.99818
sigma_a:0:0.00000000, sigma_s:0:0.01772542, mu_a:0:0.00982541, rho_a:0:0.01054125, rho_s:0:0.01913531, kappa_a:0:0.31098076
Time taken: 25.70s

Start of epoch 495
Training observations acc over epoch: 0.0002
total loss: 1.56413, total regularisd loss: 0.74592
obs u loss: 0.35319, obs v loss: 0.13065
pde u loss: 0.00072, pde v loss: 0.08075
lambda obs u: 1.47491, lambda obs v: 0.56305
lambda pde u: 0.00073, lambda pde v: 1.96131
sigma_a:0:0.00000000, sigma_s:0:0.01735825, mu_a:0:0.00976734, rho_a:0:0.01060954, rho_s:0:0.01994805, kappa_a:0:0.30856900
Time taken: 25.72s

Start of epoch 496
Training observations acc over epoch: 0.0002
total loss: 1.52614, total regularisd loss: 0.78086
obs u loss: 0.37222, obs v loss: 0.1307

In [13]:
pinn2 = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss2 = ASDM(dtype=tf.float64, D_a = 1.0, D_s = 1.0)

In [14]:
model2 = tu.TINN(pinn2, pde_loss2)

In [15]:
results2 = model2.train(epochs = 500,
                      batch_size = 512,
                      X = obs_X2,
                      Y = obs_Y2,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 8.8510
total loss: 31310.14008, total regularisd loss: 31310.14008
obs u loss: 1537.59828, obs v loss: 16589.19334
pde u loss: 427.60600, pde v loss: 12755.74245
lambda obs u: 1.00000, lambda obs v: 1.00000
lambda pde u: 1.00000, lambda pde v: 1.00000
sigma_a:0:10.05552814, sigma_s:0:8.37776684, mu_a:0:9.72718030, rho_a:0:10.42478945, rho_s:0:11.81540803, kappa_a:0:8.06830649
Time taken: 47.52s

Start of epoch 1
Training observations acc over epoch: 5.8540
total loss: 16381.38788, total regularisd loss: 16381.38788
obs u loss: 1566.74193, obs v loss: 10422.23666
pde u loss: 1.09619, pde v loss: 4391.31310
lambda obs u: 0.78156, lambda obs v: 1.97274
lambda pde u: 0.50000, lambda pde v: 0.74570
sigma_a:0:10.12764641, sigma_s:0:7.52797738, mu_a:0:9.56899632, rho_a:0:10.65753475, rho_s:0:12.65178121, kappa_a:0:7.08810265
Time taken: 31.21s

Start of epoch 2
Training observations acc over epoch: 2.3697
total loss: 12814.04734, total r

Training observations acc over epoch: 0.3971
total loss: 4776.91483, total regularisd loss: 1802.84286
obs u loss: 383.91105, obs v loss: 429.44850
pde u loss: 506.57259, pde v loss: 710.61570
lambda obs u: 2.31058, lambda obs v: 0.43889
lambda pde u: 0.41773, lambda pde v: 0.83280
sigma_a:0:2.96830678, sigma_s:0:0.37876639, mu_a:0:3.32760382, rho_a:0:17.56730250, rho_s:0:8.52301749, kappa_a:0:1.62719589
Time taken: 25.28s

Start of epoch 20
Training observations acc over epoch: 0.3024
total loss: 2716.81676, total regularisd loss: 1264.90558
obs u loss: 235.14338, obs v loss: 384.24120
pde u loss: 525.95870, pde v loss: 400.14346
lambda obs u: 1.78248, lambda obs v: 0.75367
lambda pde u: 0.60566, lambda pde v: 0.85818
sigma_a:0:1.72612756, sigma_s:0:0.38941051, mu_a:0:2.32042436, rho_a:0:18.38191688, rho_s:0:7.54614641, kappa_a:0:1.52861105
Time taken: 25.27s

Start of epoch 21
Training observations acc over epoch: 0.2180
total loss: 1130.32598, total regularisd loss: 827.56943
obs u 

Training observations acc over epoch: 0.0252
total loss: 79.49834, total regularisd loss: 111.73199
obs u loss: 34.07195, obs v loss: 17.48946
pde u loss: 34.42779, pde v loss: 3.05198
lambda obs u: 1.37635, lambda obs v: 0.72167
lambda pde u: 1.54185, lambda pde v: 0.36012
sigma_a:0:0.00000000, sigma_s:0:0.01373197, mu_a:0:0.00000000, rho_a:0:6.43809408, rho_s:0:0.54814350, kappa_a:0:12.22106147
Time taken: 25.01s

Start of epoch 39
Training observations acc over epoch: 0.0229
total loss: 72.14536, total regularisd loss: 106.26985
obs u loss: 32.40728, obs v loss: 14.56696
pde u loss: 32.56294, pde v loss: 2.62717
lambda obs u: 1.41976, lambda obs v: 0.71395
lambda pde u: 1.55581, lambda pde v: 0.31048
sigma_a:0:0.00000000, sigma_s:0:0.01500408, mu_a:0:0.00000000, rho_a:0:5.75324229, rho_s:0:0.60304479, kappa_a:0:12.34624810
Time taken: 25.15s

Start of epoch 40
Training observations acc over epoch: 0.0211
total loss: 66.45731, total regularisd loss: 101.04811
obs u loss: 30.66861, ob

Training observations acc over epoch: 0.0071
total loss: 21.96012, total regularisd loss: 37.39261
obs u loss: 12.19448, obs v loss: 2.26792
pde u loss: 8.39815, pde v loss: 0.79650
lambda obs u: 2.38953, lambda obs v: 0.39632
lambda pde u: 0.93195, lambda pde v: 0.28220
sigma_a:0:0.03036593, sigma_s:0:0.00429550, mu_a:0:0.27527457, rho_a:0:0.37489632, rho_s:0:0.01213733, kappa_a:0:0.00532888
Time taken: 25.35s

Start of epoch 58
Training observations acc over epoch: 0.0069
total loss: 22.32799, total regularisd loss: 36.90720
obs u loss: 11.83353, obs v loss: 2.31804
pde u loss: 8.02600, pde v loss: 0.82256
lambda obs u: 2.43050, lambda obs v: 0.41937
lambda pde u: 0.87240, lambda pde v: 0.27774
sigma_a:0:0.03898545, sigma_s:0:0.00200529, mu_a:0:0.32977157, rho_a:0:0.42481614, rho_s:0:0.01324507, kappa_a:0:0.00501296
Time taken: 25.25s

Start of epoch 59
Training observations acc over epoch: 0.0067
total loss: 21.77470, total regularisd loss: 35.76822
obs u loss: 11.46805, obs v loss:

Training observations acc over epoch: 0.0057
total loss: 22.64834, total regularisd loss: 32.41537
obs u loss: 10.00162, obs v loss: 1.68634
pde u loss: 6.37671, pde v loss: 0.56442
lambda obs u: 2.86612, lambda obs v: 0.43252
lambda pde u: 0.47254, lambda pde v: 0.22881
sigma_a:0:0.10054435, sigma_s:0:0.00669285, mu_a:0:0.71257397, rho_a:0:0.78600360, rho_s:0:0.01689746, kappa_a:0:0.00721079
Time taken: 25.52s

Start of epoch 77
Training observations acc over epoch: 0.0056
total loss: 23.29575, total regularisd loss: 32.18102
obs u loss: 9.87538, obs v loss: 1.65869
pde u loss: 6.41630, pde v loss: 0.55761
lambda obs u: 2.88969, lambda obs v: 0.42632
lambda pde u: 0.45527, lambda pde v: 0.22872
sigma_a:0:0.10442945, sigma_s:0:0.00715142, mu_a:0:0.72031759, rho_a:0:0.79429335, rho_s:0:0.01700235, kappa_a:0:0.00752733
Time taken: 25.12s

Start of epoch 78
Training observations acc over epoch: 0.0056
total loss: 23.80578, total regularisd loss: 32.32572
obs u loss: 9.88844, obs v loss: 1

...

Training observations acc over epoch: 0.0012
total loss: 3.46973, total regularisd loss: 4.72106
obs u loss: 1.89743, obs v loss: 0.53519
pde u loss: 1.14602, pde v loss: 0.13353
lambda obs u: 1.71206, lambda obs v: 1.08245
lambda pde u: 0.72368, lambda pde v: 0.48181
sigma_a:0:0.04621859, sigma_s:0:0.01055614, mu_a:0:0.53443005, rho_a:0:0.61567268, rho_s:0:0.02007966, kappa_a:0:0.00000000
Time taken: 25.77s

Start of epoch 419
Training observations acc over epoch: 0.0012
total loss: 3.43575, total regularisd loss: 4.64535
obs u loss: 1.85778, obs v loss: 0.52316
pde u loss: 1.15403, pde v loss: 0.13137
lambda obs u: 1.69753, lambda obs v: 1.07772
lambda pde u: 0.74516, lambda pde v: 0.47959
sigma_a:0:0.04593031, sigma_s:0:0.01142506, mu_a:0:0.53504443, rho_a:0:0.61503408, rho_s:0:0.02065402, kappa_a:0:0.00000000
Time taken: 26.19s

Start of epoch 420
Training observations acc over epoch: 0.0013
total loss: 3.53610, total regularisd loss: 5.03044
obs u loss: 2.08362, obs v loss: 0.5356

Training observations acc over epoch: 0.0012
total loss: 3.40218, total regularisd loss: 4.73504
obs u loss: 1.88620, obs v loss: 0.52559
pde u loss: 1.12429, pde v loss: 0.13095
lambda obs u: 1.72391, lambda obs v: 1.07452
lambda pde u: 0.72838, lambda pde v: 0.47319
sigma_a:0:0.04438522, sigma_s:0:0.01104372, mu_a:0:0.52345198, rho_a:0:0.60535257, rho_s:0:0.02068750, kappa_a:0:0.00004768
Time taken: 26.74s

Start of epoch 438
Training observations acc over epoch: 0.0012
total loss: 3.34409, total regularisd loss: 4.62855
obs u loss: 1.86262, obs v loss: 0.51923
pde u loss: 1.09505, pde v loss: 0.13107
lambda obs u: 1.70658, lambda obs v: 1.09300
lambda pde u: 0.73145, lambda pde v: 0.46897
sigma_a:0:0.04394450, sigma_s:0:0.01137442, mu_a:0:0.52252167, rho_a:0:0.60502400, rho_s:0:0.02066753, kappa_a:0:0.00002731
Time taken: 26.12s

Start of epoch 439
Training observations acc over epoch: 0.0012
total loss: 3.41046, total regularisd loss: 4.72736
obs u loss: 1.91990, obs v loss: 0.5225

Training observations acc over epoch: 0.0012
total loss: 3.42036, total regularisd loss: 4.73707
obs u loss: 1.91986, obs v loss: 0.52039
pde u loss: 1.13926, pde v loss: 0.12877
lambda obs u: 1.72909, lambda obs v: 1.07061
lambda pde u: 0.73599, lambda pde v: 0.46431
sigma_a:0:0.03943713, sigma_s:0:0.01130083, mu_a:0:0.51210069, rho_a:0:0.59717395, rho_s:0:0.02066370, kappa_a:0:0.00007307
Time taken: 26.36s

Start of epoch 457
Training observations acc over epoch: 0.0012
total loss: 3.42093, total regularisd loss: 4.84023
obs u loss: 1.96581, obs v loss: 0.52146
pde u loss: 1.11833, pde v loss: 0.12883
lambda obs u: 1.74986, lambda obs v: 1.05165
lambda pde u: 0.73963, lambda pde v: 0.45885
sigma_a:0:0.04220443, sigma_s:0:0.01038675, mu_a:0:0.51240594, rho_a:0:0.59776705, rho_s:0:0.02020431, kappa_a:0:0.00016694
Time taken: 26.59s

Start of epoch 458
Training observations acc over epoch: 0.0011
total loss: 3.19470, total regularisd loss: 4.45969
obs u loss: 1.76588, obs v loss: 0.5045

Training observations acc over epoch: 0.0012
total loss: 3.25649, total regularisd loss: 4.58970
obs u loss: 1.87303, obs v loss: 0.51455
pde u loss: 1.07705, pde v loss: 0.12525
lambda obs u: 1.71486, lambda obs v: 1.05344
lambda pde u: 0.76485, lambda pde v: 0.46684
sigma_a:0:0.03953192, sigma_s:0:0.01127791, mu_a:0:0.50112463, rho_a:0:0.58974743, rho_s:0:0.02058059, kappa_a:0:0.00000000
Time taken: 25.76s

Start of epoch 476
Training observations acc over epoch: 0.0012
total loss: 3.27404, total regularisd loss: 4.62802
obs u loss: 1.86294, obs v loss: 0.50969
pde u loss: 1.09410, pde v loss: 0.12763
lambda obs u: 1.72243, lambda obs v: 1.04434
lambda pde u: 0.76234, lambda pde v: 0.47090
sigma_a:0:0.03777570, sigma_s:0:0.01159696, mu_a:0:0.50055385, rho_a:0:0.58828338, rho_s:0:0.02068125, kappa_a:0:0.00004380
Time taken: 26.17s

Start of epoch 477
Training observations acc over epoch: 0.0013
total loss: 3.53208, total regularisd loss: 5.12701
obs u loss: 2.10716, obs v loss: 0.5207

Training observations acc over epoch: 0.0013
total loss: 3.38313, total regularisd loss: 4.96774
obs u loss: 2.04374, obs v loss: 0.52348
pde u loss: 1.10022, pde v loss: 0.12650
lambda obs u: 1.76080, lambda obs v: 1.02345
lambda pde u: 0.76313, lambda pde v: 0.45262
sigma_a:0:0.03592960, sigma_s:0:0.01055664, mu_a:0:0.49306368, rho_a:0:0.58207573, rho_s:0:0.02040150, kappa_a:0:0.00011920
Time taken: 26.76s

Start of epoch 495
Training observations acc over epoch: 0.0011
total loss: 3.15416, total regularisd loss: 4.56474
obs u loss: 1.82183, obs v loss: 0.49965
pde u loss: 1.03361, pde v loss: 0.12531
lambda obs u: 1.72451, lambda obs v: 1.04830
lambda pde u: 0.75003, lambda pde v: 0.47717
sigma_a:0:0.03694587, sigma_s:0:0.01096908, mu_a:0:0.49183569, rho_a:0:0.58207249, rho_s:0:0.02048308, kappa_a:0:0.00000000
Time taken: 26.24s

Start of epoch 496
Training observations acc over epoch: 0.0011
total loss: 3.15214, total regularisd loss: 4.46042
obs u loss: 1.80210, obs v loss: 0.4986

In [21]:
for para in pde_loss2.trainables():
    print(f"{para.name}: {para.numpy()[0]}")

sigma_a:0: 0.0343791316457888
sigma_s:0: 0.011075548714052972
mu_a:0: 0.4916876608365929
rho_a:0: 0.5803898009306196
rho_s:0: 0.020798560846101613
kappa_a:0: 2.312989778952719e-05


In [22]:
results2_2 = model2.train(epochs = 100,
                      batch_size = 512,
                      X = obs_X2,
                      Y = obs_Y2,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 0.0012
total loss: 3.63669, total regularisd loss: 4.74446
obs u loss: 1.93022, obs v loss: 0.50409
pde u loss: 1.07750, pde v loss: 0.12488
lambda obs u: 1.73620, lambda obs v: 1.03175
lambda pde u: 0.75503, lambda pde v: 0.47702
sigma_a:0:0.03563372, sigma_s:0:0.01074451, mu_a:0:0.48955325, rho_a:0:0.57943883, rho_s:0:0.02064145, kappa_a:0:0.00000000
Time taken: 26.84s

Start of epoch 1
Training observations acc over epoch: 0.0012
total loss: 3.23431, total regularisd loss: 4.63698
obs u loss: 1.87610, obs v loss: 0.50403
pde u loss: 1.05972, pde v loss: 0.12482
lambda obs u: 1.70895, lambda obs v: 1.04031
lambda pde u: 0.77421, lambda pde v: 0.47653
sigma_a:0:0.03547816, sigma_s:0:0.01112554, mu_a:0:0.48984816, rho_a:0:0.57998195, rho_s:0:0.02034138, kappa_a:0:0.00000000
Time taken: 26.33s

Start of epoch 2
Training observations acc over epoch: 0.0011
total loss: 3.07748, total regularisd loss: 4.34759
obs u loss: 1.74630, obs 

Training observations acc over epoch: 0.0011
total loss: 3.07129, total regularisd loss: 4.39101
obs u loss: 1.75621, obs v loss: 0.48825
pde u loss: 1.01674, pde v loss: 0.12527
lambda obs u: 1.72991, lambda obs v: 1.04154
lambda pde u: 0.75366, lambda pde v: 0.47490
sigma_a:0:0.03419996, sigma_s:0:0.01167817, mu_a:0:0.48294069, rho_a:0:0.57436272, rho_s:0:0.02108386, kappa_a:0:0.00000000
Time taken: 26.57s

Start of epoch 20
Training observations acc over epoch: 0.0011
total loss: 3.17180, total regularisd loss: 4.54628
obs u loss: 1.84681, obs v loss: 0.49588
pde u loss: 1.02949, pde v loss: 0.12449
lambda obs u: 1.73814, lambda obs v: 1.02553
lambda pde u: 0.76517, lambda pde v: 0.47116
sigma_a:0:0.03315772, sigma_s:0:0.01083626, mu_a:0:0.48235402, rho_a:0:0.57420230, rho_s:0:0.02062034, kappa_a:0:0.00000000
Time taken: 27.53s

Start of epoch 21
Training observations acc over epoch: 0.0011
total loss: 3.12997, total regularisd loss: 4.46523
obs u loss: 1.78961, obs v loss: 0.48646


Training observations acc over epoch: 0.0011
total loss: 2.97077, total regularisd loss: 4.23303
obs u loss: 1.69905, obs v loss: 0.48079
pde u loss: 0.98923, pde v loss: 0.12437
lambda obs u: 1.69191, lambda obs v: 1.02491
lambda pde u: 0.79067, lambda pde v: 0.49251
sigma_a:0:0.03291376, sigma_s:0:0.01132821, mu_a:0:0.47476310, rho_a:0:0.56723968, rho_s:0:0.02061015, kappa_a:0:0.00000000
Time taken: 26.42s

Start of epoch 40
Training observations acc over epoch: 0.0012
total loss: 3.26198, total regularisd loss: 4.85443
obs u loss: 2.04405, obs v loss: 0.50054
pde u loss: 1.04021, pde v loss: 0.12306
lambda obs u: 1.80000, lambda obs v: 0.98316
lambda pde u: 0.76285, lambda pde v: 0.45398
sigma_a:0:0.03286097, sigma_s:0:0.01133676, mu_a:0:0.47505443, rho_a:0:0.56791520, rho_s:0:0.02033249, kappa_a:0:0.00000000
Time taken: 26.77s

Start of epoch 41
Training observations acc over epoch: 0.0011
total loss: 3.03303, total regularisd loss: 4.45029
obs u loss: 1.75856, obs v loss: 0.48621


Training observations acc over epoch: 0.0012
total loss: 3.30588, total regularisd loss: 4.79574
obs u loss: 2.00828, obs v loss: 0.49696
pde u loss: 1.06877, pde v loss: 0.12287
lambda obs u: 1.76382, lambda obs v: 0.98685
lambda pde u: 0.77987, lambda pde v: 0.46946
sigma_a:0:0.03091283, sigma_s:0:0.01197160, mu_a:0:0.46809329, rho_a:0:0.56240242, rho_s:0:0.02041896, kappa_a:0:0.00000000
Time taken: 27.76s

Start of epoch 60
Training observations acc over epoch: 0.0011
total loss: 2.98187, total regularisd loss: 4.36336
obs u loss: 1.74152, obs v loss: 0.48144
pde u loss: 0.97428, pde v loss: 0.12082
lambda obs u: 1.71544, lambda obs v: 1.01652
lambda pde u: 0.77930, lambda pde v: 0.48874
sigma_a:0:0.03142676, sigma_s:0:0.01111556, mu_a:0:0.46651712, rho_a:0:0.56195088, rho_s:0:0.02067864, kappa_a:0:0.00007199
Time taken: 27.42s

Start of epoch 61
Training observations acc over epoch: 0.0011
total loss: 2.91157, total regularisd loss: 4.16072
obs u loss: 1.67597, obs v loss: 0.48112


Training observations acc over epoch: 0.0011
total loss: 3.10086, total regularisd loss: 4.66490
obs u loss: 1.83742, obs v loss: 0.48830
pde u loss: 0.97473, pde v loss: 0.12052
lambda obs u: 1.78195, lambda obs v: 0.99928
lambda pde u: 0.76107, lambda pde v: 0.45771
sigma_a:0:0.03107398, sigma_s:0:0.01057373, mu_a:0:0.46147560, rho_a:0:0.55828987, rho_s:0:0.02052571, kappa_a:0:0.00000000
Time taken: 26.80s

Start of epoch 80
Training observations acc over epoch: 0.0010
total loss: 2.89108, total regularisd loss: 4.13558
obs u loss: 1.62125, obs v loss: 0.46890
pde u loss: 0.94952, pde v loss: 0.12102
lambda obs u: 1.70982, lambda obs v: 1.02958
lambda pde u: 0.77136, lambda pde v: 0.48924
sigma_a:0:0.02978266, sigma_s:0:0.01051719, mu_a:0:0.46145177, rho_a:0:0.55756162, rho_s:0:0.02045575, kappa_a:0:0.00011904
Time taken: 27.10s

Start of epoch 81
Training observations acc over epoch: 0.0012
total loss: 3.13497, total regularisd loss: 4.55563
obs u loss: 1.87893, obs v loss: 0.48729


Training observations acc over epoch: 0.0011
total loss: 3.08297, total regularisd loss: 4.53776
obs u loss: 1.85708, obs v loss: 0.47869
pde u loss: 1.00325, pde v loss: 0.12075
lambda obs u: 1.76655, lambda obs v: 0.97348
lambda pde u: 0.78850, lambda pde v: 0.47146
sigma_a:0:0.02797015, sigma_s:0:0.01144193, mu_a:0:0.45627132, rho_a:0:0.55240556, rho_s:0:0.02038532, kappa_a:0:0.00011865
Time taken: 27.32s


In [23]:
for para in pde_loss2.trainables():
    print(f"{para.name}: {para.numpy()[0]}")

sigma_a:0: 0.027970148949427558
sigma_s:0: 0.011441933132905835
mu_a:0: 0.45627131561452355
rho_a:0: 0.5524055565641843
rho_s:0: 0.020385315862480675
kappa_a:0: 0.00011865479156700788


In [24]:
pinn3 = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss3 = ASDM(dtype=tf.float64, D_a = 1.0, D_s = 4.0)

In [25]:
model3 = tu.TINN(pinn3, pde_loss3)

In [ ]:
results3 = model3.train(epochs = 500,
                      batch_size = 512,
                      X = obs_X2,
                      Y = obs_Y2,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 8.8021
total loss: 31032.76525, total regularisd loss: 31032.76525
obs u loss: 1530.55521, obs v loss: 16496.17170
pde u loss: 466.76271, pde v loss: 12539.27564
lambda obs u: 1.00000, lambda obs v: 1.00000
lambda pde u: 1.00000, lambda pde v: 1.00000
sigma_a:0:10.05005491, sigma_s:0:8.38077383, mu_a:0:9.76782146, rho_a:0:10.34804045, rho_s:0:11.86291525, kappa_a:0:8.03814692
Time taken: 50.47s

Start of epoch 1
Training observations acc over epoch: 5.8171
total loss: 16211.61730, total regularisd loss: 16211.61730
obs u loss: 1552.07241, obs v loss: 10361.36601
pde u loss: 1.20419, pde v loss: 4296.97469
lambda obs u: 0.79394, lambda obs v: 1.96958
lambda pde u: 0.50000, lambda pde v: 0.73647
sigma_a:0:10.12610737, sigma_s:0:7.53224694, mu_a:0:9.61984442, rho_a:0:10.56274815, rho_s:0:12.69782776, kappa_a:0:7.05952550
Time taken: 33.74s

Start of epoch 2
Training observations acc over epoch: 2.3446
total loss: 12841.00384, total r

Training observations acc over epoch: 0.3920
total loss: 9812.62412, total regularisd loss: 2106.25308
obs u loss: 474.88512, obs v loss: 327.96130
pde u loss: 413.77241, pde v loss: 1015.93972
lambda obs u: 2.66899, lambda obs v: 0.17944
lambda pde u: 0.41779, lambda pde v: 0.73377
sigma_a:0:2.91596953, sigma_s:0:0.68026110, mu_a:0:3.64776424, rho_a:0:16.48610592, rho_s:0:10.15338291, kappa_a:0:1.00252107
Time taken: 26.19s

Start of epoch 20
Training observations acc over epoch: 0.3109
total loss: 4761.06769, total regularisd loss: 1443.55400
obs u loss: 253.71205, obs v loss: 382.91884
pde u loss: 764.25791, pde v loss: 515.67058
lambda obs u: 2.29517, lambda obs v: 0.55451
lambda pde u: 0.53254, lambda pde v: 0.61778
sigma_a:0:1.86176659, sigma_s:0:0.45876414, mu_a:0:2.42829506, rho_a:0:17.38376919, rho_s:0:9.10192179, kappa_a:0:0.58136984
Time taken: 26.33s

Start of epoch 21
Training observations acc over epoch: 0.2327
total loss: 1996.67015, total regularisd loss: 882.52119
obs 

Training observations acc over epoch: 0.0224
total loss: 75.20889, total regularisd loss: 107.54137
obs u loss: 31.97645, obs v loss: 13.97392
pde u loss: 33.04701, pde v loss: 3.74889
lambda obs u: 1.49638, lambda obs v: 0.65173
lambda pde u: 1.55890, lambda pde v: 0.29299
sigma_a:0:0.00000000, sigma_s:0:0.01209103, mu_a:0:0.00000000, rho_a:0:5.14099579, rho_s:0:0.90065769, kappa_a:0:9.71448766
Time taken: 25.91s

Start of epoch 39
Training observations acc over epoch: 0.0199
total loss: 67.65796, total regularisd loss: 99.68333
obs u loss: 29.37337, obs v loss: 11.46349
pde u loss: 30.35283, pde v loss: 3.21364
lambda obs u: 1.56176, lambda obs v: 0.66594
lambda pde u: 1.52549, lambda pde v: 0.24681
sigma_a:0:0.00000000, sigma_s:0:0.01727472, mu_a:0:0.00000000, rho_a:0:4.45574920, rho_s:0:1.01533427, kappa_a:0:9.86323978
Time taken: 26.33s

Start of epoch 40
Training observations acc over epoch: 0.0179
total loss: 61.42555, total regularisd loss: 93.61750
obs u loss: 27.39980, obs v 

Training observations acc over epoch: 0.0068
total loss: 25.06646, total regularisd loss: 36.72048
obs u loss: 11.73125, obs v loss: 2.14833
pde u loss: 7.87930, pde v loss: 1.75814
lambda obs u: 2.43356, lambda obs v: 0.41551
lambda pde u: 0.85833, lambda pde v: 0.29260
sigma_a:0:0.04790390, sigma_s:0:0.02364050, mu_a:0:0.41065896, rho_a:0:0.44821942, rho_s:0:0.04559422, kappa_a:0:0.00094819
Time taken: 26.18s

Start of epoch 58
Training observations acc over epoch: 0.0067
total loss: 24.71887, total regularisd loss: 36.03718
obs u loss: 11.51200, obs v loss: 2.11650
pde u loss: 7.73468, pde v loss: 1.72145
lambda obs u: 2.45715, lambda obs v: 0.42210
lambda pde u: 0.82123, lambda pde v: 0.29952
sigma_a:0:0.05267363, sigma_s:0:0.02420566, mu_a:0:0.44146906, rho_a:0:0.47531709, rho_s:0:0.04766666, kappa_a:0:0.00105303
Time taken: 26.12s

Start of epoch 59
Training observations acc over epoch: 0.0066
total loss: 24.51410, total regularisd loss: 35.58303
obs u loss: 11.36650, obs v loss:

Training observations acc over epoch: 0.0058
total loss: 24.59955, total regularisd loss: 31.54881
obs u loss: 9.98618, obs v loss: 1.80833
pde u loss: 6.70852, pde v loss: 1.44951
lambda obs u: 2.69049, lambda obs v: 0.45900
lambda pde u: 0.51545, lambda pde v: 0.33505
sigma_a:0:0.09035348, sigma_s:0:0.03630530, mu_a:0:0.68819712, rho_a:0:0.70967554, rho_s:0:0.05998898, kappa_a:0:0.00314015
Time taken: 26.02s

Start of epoch 77
Training observations acc over epoch: 0.0058
total loss: 25.36657, total regularisd loss: 32.06056
obs u loss: 10.11041, obs v loss: 1.80220
pde u loss: 6.87464, pde v loss: 1.45590
lambda obs u: 2.69358, lambda obs v: 0.46152
lambda pde u: 0.51123, lambda pde v: 0.33367
sigma_a:0:0.09323382, sigma_s:0:0.03703835, mu_a:0:0.69448568, rho_a:0:0.71534559, rho_s:0:0.06055034, kappa_a:0:0.00327333
Time taken: 26.01s

Start of epoch 78
Training observations acc over epoch: 0.0057
total loss: 24.95689, total regularisd loss: 31.58176
obs u loss: 9.97143, obs v loss: 1

...

Training observations acc over epoch: 0.0034
total loss: 17.77780, total regularisd loss: 18.59062
obs u loss: 5.84064, obs v loss: 1.20715
pde u loss: 4.21658, pde v loss: 0.99642
lambda obs u: 2.72073, lambda obs v: 0.50206
lambda pde u: 0.39648, lambda pde v: 0.38073
sigma_a:0:0.07796855, sigma_s:0:0.04935708, mu_a:0:0.71121923, rho_a:0:0.73662276, rho_s:0:0.07505649, kappa_a:0:0.00046463
Time taken: 26.54s

Start of epoch 153
Training observations acc over epoch: 0.0034
total loss: 17.68178, total regularisd loss: 18.24459
obs u loss: 5.73456, obs v loss: 1.19757
pde u loss: 4.19084, pde v loss: 0.99695
lambda obs u: 2.69967, lambda obs v: 0.50910
lambda pde u: 0.40380, lambda pde v: 0.38743
sigma_a:0:0.07538132, sigma_s:0:0.04961452, mu_a:0:0.70963346, rho_a:0:0.73359095, rho_s:0:0.07417917, kappa_a:0:0.00039643
Time taken: 26.52s

Start of epoch 154
Training observations acc over epoch: 0.0034
total loss: 17.26243, total regularisd loss: 17.99702
obs u loss: 5.67973, obs v loss: 

Training observations acc over epoch: 0.0028
total loss: 13.22445, total regularisd loss: 14.05486
obs u loss: 4.61640, obs v loss: 1.10030
pde u loss: 3.41764, pde v loss: 0.87837
lambda obs u: 2.46267, lambda obs v: 0.65044
lambda pde u: 0.44769, lambda pde v: 0.43919
sigma_a:0:0.06506546, sigma_s:0:0.04908203, mu_a:0:0.66914347, rho_a:0:0.70625968, rho_s:0:0.07760151, kappa_a:0:0.00002097
Time taken: 25.97s

Start of epoch 172
Training observations acc over epoch: 0.0028
total loss: 13.27117, total regularisd loss: 14.00650
obs u loss: 4.61339, obs v loss: 1.09813
pde u loss: 3.44956, pde v loss: 0.88031
lambda obs u: 2.45652, lambda obs v: 0.65034
lambda pde u: 0.45002, lambda pde v: 0.44311
sigma_a:0:0.06494090, sigma_s:0:0.04973863, mu_a:0:0.66693933, rho_a:0:0.70454819, rho_s:0:0.07799971, kappa_a:0:0.00002799
Time taken: 25.59s

Start of epoch 173
Training observations acc over epoch: 0.0028
total loss: 13.08373, total regularisd loss: 13.85616
obs u loss: 4.56917, obs v loss: 

Training observations acc over epoch: 0.0024
total loss: 10.82215, total regularisd loss: 11.39471
obs u loss: 3.89810, obs v loss: 1.02078
pde u loss: 2.89597, pde v loss: 0.78090
lambda obs u: 2.27063, lambda obs v: 0.77728
lambda pde u: 0.47441, lambda pde v: 0.47768
sigma_a:0:0.06010526, sigma_s:0:0.04798972, mu_a:0:0.63692807, rho_a:0:0.68801098, rho_s:0:0.08041363, kappa_a:0:0.00000000
Time taken: 25.63s

Start of epoch 191
Training observations acc over epoch: 0.0024
total loss: 10.65164, total regularisd loss: 11.33153
obs u loss: 3.88306, obs v loss: 1.01914
pde u loss: 2.85120, pde v loss: 0.77403
lambda obs u: 2.25446, lambda obs v: 0.79060
lambda pde u: 0.47638, lambda pde v: 0.47856
sigma_a:0:0.05985409, sigma_s:0:0.04900343, mu_a:0:0.63605318, rho_a:0:0.68791528, rho_s:0:0.08081879, kappa_a:0:0.00000000
Time taken: 25.43s

Start of epoch 192
Training observations acc over epoch: 0.0024
total loss: 10.44693, total regularisd loss: 11.14576
obs u loss: 3.83577, obs v loss: 

Training observations acc over epoch: 0.0021
total loss: 9.14921, total regularisd loss: 9.58537
obs u loss: 3.37173, obs v loss: 0.95842
pde u loss: 2.41911, pde v loss: 0.69817
lambda obs u: 2.13027, lambda obs v: 0.89911
lambda pde u: 0.47333, lambda pde v: 0.49729
sigma_a:0:0.05453389, sigma_s:0:0.04942304, mu_a:0:0.61583380, rho_a:0:0.67582438, rho_s:0:0.08269843, kappa_a:0:0.00002005
Time taken: 25.24s

Start of epoch 210
Training observations acc over epoch: 0.0022
total loss: 9.30960, total regularisd loss: 9.78140
obs u loss: 3.47653, obs v loss: 0.95792
pde u loss: 2.46349, pde v loss: 0.70003
lambda obs u: 2.12536, lambda obs v: 0.90759
lambda pde u: 0.47260, lambda pde v: 0.49445
sigma_a:0:0.05384700, sigma_s:0:0.04824799, mu_a:0:0.61318700, rho_a:0:0.67614150, rho_s:0:0.08392333, kappa_a:0:0.00000000
Time taken: 25.11s

Start of epoch 211
Training observations acc over epoch: 0.0021
total loss: 9.17473, total regularisd loss: 9.57988
obs u loss: 3.39990, obs v loss: 0.9557

Training observations acc over epoch: 0.0020
total loss: 8.26078, total regularisd loss: 8.58967
obs u loss: 3.12366, obs v loss: 0.91917
pde u loss: 2.17780, pde v loss: 0.64395
lambda obs u: 2.00950, lambda obs v: 1.00071
lambda pde u: 0.47708, lambda pde v: 0.51272
sigma_a:0:0.05099489, sigma_s:0:0.04771847, mu_a:0:0.59199218, rho_a:0:0.66386062, rho_s:0:0.08326114, kappa_a:0:0.00016736
Time taken: 25.30s

Start of epoch 229
Training observations acc over epoch: 0.0020
total loss: 8.41908, total regularisd loss: 8.65171
obs u loss: 3.15498, obs v loss: 0.91596
pde u loss: 2.22727, pde v loss: 0.64868
lambda obs u: 2.01336, lambda obs v: 0.99924
lambda pde u: 0.47683, lambda pde v: 0.51056
sigma_a:0:0.04990454, sigma_s:0:0.04836144, mu_a:0:0.59074284, rho_a:0:0.66225012, rho_s:0:0.08490289, kappa_a:0:0.00000000
Time taken: 25.32s

Start of epoch 230
Training observations acc over epoch: 0.0020
total loss: 8.27452, total regularisd loss: 8.51958
obs u loss: 3.10067, obs v loss: 0.9117